<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_0210ReadingAloudModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語の処理と文脈および意味表現プロジェクト 資料

- Project for Word processings, context, and semantic representation (WordProc&Con_Sem)
- date: 2023_0210
- author: 浅川伸一 <asakawa@ieee.org>

- `git` コマンドを使ってソースコードをローカルディスクに保存するなどしてください。

```bash
git clone https://github.com/ShinAsakawa/RAM.git
```


## おあそび RAM (abbriviated for Reading Aloud Model) の描画

In [ ]:
# ここはお遊びなので，スキップしても良い
import IPython
#lams_url="https://livedoor.blogimg.jp/ftb001/imgs/b/4/b4629a79.jpg"
#lams_url="https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp"
#IPython.display.Image(url=lams_url)

import os
lum_img_fname = 'lum@pc.webp'
if not os.path.exists(lum_img_fname):
    !wget "https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp"


import matplotlib.pyplot as plt
x = plt.imread('lum@pc.webp')
plt.figure(figsize=(4,7))
plt.axis('off')
plt.imshow(x)

### 0.0.1 RNN, LSTM, GRU に関して
<!--
次のサンプルコードを参照

```python
_gru = nn.GRU(input_size=3, hidden_size=2, num_layers=1, batch_first=False)
#_gru(input=torch.tensor([1,2,3]), h0=torch.tensor([4,5]))
#help(nn.GRU)

seq_len, n_seq, n_inp, n_hid, n_layers = 5, 3, 10, 8, 2
seq_len, n_seq, n_inp, n_hid, n_layers = 1, 3, 5, 7, 2

inp_sf = torch.randn(seq_len, n_seq, n_inp)     # sequence first (=time) first
inp_bf = torch.randn(n_seq, seq_len, n_inp)     # batch (=sentences order) first
hid0   = torch.randn(n_layers, n_seq, n_hid)
cel0   = torch.randn(n_layers, n_seq, n_hid)

model0 = nn.GRU(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=False)
model1 = nn.GRU(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=True)
model2 = nn.RNN(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=False)
model3 = nn.RNN(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=True)
model4 = nn.LSTM(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=False)
model5 = nn.LSTM(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=True)
model6 = nn.LSTM(input_size=n_inp, hidden_size=n_hid, num_layers=n_layers, batch_first=False)

output0, hn = model0(inp_sf, hid0)
print(f'output0.size():{output0.size()}', f'hn.size():{hn.size()}')

output1, hn = model1(inp_bf, hid0)
print(f'output1.size():{output1.size()}', f'hn.size():{hn.size()}')

output2, hn = model2(inp_sf, hid0)
print(f'output2.size():{output2.size()}', f'hn.size():{hn.size()}')

output3, hn = model3(inp_bf, hid0)
print(f'output3.size():{output3.size()}', f'hn.size():{hn.size()}')

output4, (hn, cn) = model4(inp_sf, (hid0, cel0))
print(f'output4.size():{output4.size()}', f'hn.size():{hn.size()}')

output5, (hn, cn) = model5(inp_bf, (hid0, cel0))
print(f'output5.size():{output5.size()}', f'hn.size():{hn.size()}')
```

出力は以下の通り

```bash
output0.size():torch.Size([1, 3, 7]) hn.size():torch.Size([2, 3, 7])
output1.size():torch.Size([3, 1, 7]) hn.size():torch.Size([2, 3, 7])
output2.size():torch.Size([1, 3, 7]) hn.size():torch.Size([2, 3, 7])
output3.size():torch.Size([3, 1, 7]) hn.size():torch.Size([2, 3, 7])
output4.size():torch.Size([1, 3, 7]) hn.size():torch.Size([2, 3, 7])
output5.size():torch.Size([3, 1, 7]) hn.size():torch.Size([2, 3, 7])
``` -->

## 0.1 2023_0120 の議論

符号化器-復号化器モデルは，[seq2seq](https://arxiv.org/abs/1409.3215) と呼ばれるモデルでもある。
邦訳すれば，系列-2-系列 モデルなのだが，今回のプロジェクトでは，意味表象が，系列ではない。
そのため，seq2seq という名称よりも，より一般化した，符号化器-復号化器 (enc-dec model) モデルとしたい。

<center>
<img src="https://raw.githubusercontent.com/komazawa-deep-learning/komazawa-deep-learning.github.io/master/assets/2014Sutskever_S22_Fig1.svg" width="66%">
<br/>    
From Sutskever 2014, Figure 1.
</center>

上図は，seq2seq モデルの概略図である。
符号化器と呼ばれる部分は，トークン `<EOS>` が入力された時点までである。
それ以降は，復号化器となる。
符号化器と復号化器とで，一時刻前の中間層の状態が共有されていることがポイントである。
seq2seq は翻訳モデルであり，符号化器と復号化器とで，言語モデルの扱う言語が異なっている。
具体的には，フランス語と英語である。


オリジナルの，三角モデルにおける o2p については，三層のニューラルネットワークとみなしうる。
このため，o2p の中間層は，識別性能を向上する役割と，モダリティ間の結合という２つの異なる役割を担っていたとみなすことができる。
理論的には，両者を分離する必要も，統合する必要もない，どちらにしても積極的な理由は存在しないと思われる。
駄菓子菓子，計算論的な役割においては，異なるモダリティ間の通信を媒介する役割と，入力モダリティにおける表象を確立するという意味合いを分離すると，役割分担が明確になるのであろうということである。

---

* 一文字の orth2phon を担保したいために，全角の数字，アルファベット，ひらがな，計 109 文字をデータ先頭に追加した。
* Fushimi1999 (Psyc. Rev.) の語彙リストを fushimi1999_list として収録
* Fushimi1999_list の扱いに伴い訓練語彙数を 10K から 20K に増加
* 学習率 lr は 0.001 だと収束しない。0.0001 であれば良好であり，訓練損失 0.01 程度，訓練精度 0.987 程度までに至る。
* ただし，一文字データセット onechar_dataset では lr=0.001 の方が収束が早い。
これは，データセットサイズが 20K と 0.1K と 20 倍の差があるためであろう。
* 近藤先生が，GPU 上で実行してくださった訓練済モデルのファイル名が `decoder256new.pt` と `encoder256new.pt` である。
これは，中間層ユニット数が 256 である orth2phon モデルの訓練済モデルである。
* `_train()` 関数内で，正解判定をする際に，GPU から CPU へ転送しなければいけないことを忘れていたので修正した。
具体的には， `detach()` と `numpy()` の間に `cpu()` を挿入した。2 箇所
```python
    ok_flag = (ok_flag) and (decoder_output.argmax() == target_tensor[di].detach().cpu().numpy()[0])
```
* 近藤先生の GPU で訓練済モデルを CPU 環境で実行する必要がある場合，変更して読み込む必要がある

```python
encoder_pretrained_fname = 'encoder256new.pt'
decoder_pretrained_fname = 'decoder256new.pt'
if os.path.exists(encoder_pretrained_fname):
    encoder = torch.load(encoder_pretrained_fname, _location=torch.device(device))map
    
if os.path.exists(decoder_pretrained_fname):
    decoder = torch.load(decoder_pretrained_fname, map_location=torch.device(device))
```

近藤先生の実験によれば，結果は以下の通りである(そうだ)。

正答率

|   | 条件 | 記述         | 正解率 | 
|:----|:-----|:------------|:------|
|WORD |   HF |1:consistent |　18/20
|WORD |   HF |2:typical    |   HF___inconsist  16/20|
|WORD |   HF |3:atypical   |   HF___atypical_  8/20 |
|WORD |   LF |1:consistent |   LF___consist__  14/20|
|WORD |   LF |2:typical    |   LF___inconsist  9/20|
|WORD |   LF |3:atypical   |   LF___atypical_  3/20|

* 伏見らではでなかったatypical効果だけでなく，
　consistent-typicalの差もある程度ある気がします
 また，LFでも効果ありであり，かつ，頻度効果もあり
* **今回，L(legitimate alternative reading of components） マークを付けてみました**
  Lm, Lnは，モーラ間違い，一文字間違いと混合

アクセプト率

|     | 条件 | 記述         | 正解率 | 
|:----|:----|:------------|:------|
|非単語| HF  | 1:consistent|HFNW_consist__  17/20|
|非単語| HF  | 2:typical   |HFNW_inconsist　　17/20|
|非単語| HF  | 3:ambiguous |HFNW_ambiguous  13/20|
|非単語| LF  | 1:consistent|LFNW_consist__  15/20|
|非単語| LF  | 2:typical   |LFNW_inconsist  13/20|
|非単語| LF  | 3:ambiguous |LFNW_ambiguous  7/20|

* かなり読めますね．アクセプトは，どんな読みでもいいので読めそうな読み方ならOKにしています．
　単語の L と同じになります．
* **結構驚きは，非単語のときに連濁や促音化ができているところ**

## 0.2 近藤先生からいただいた結果の描画

In [ ]:
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

#import matplotlib    
#matplotlib.rcParams['text.usetex'] = True    
fig, ax = plt.subplots(1,2, figsize=(8,4))

fig.suptitle('Fushimi+1999 単語リストの検証 (o2p, hid256)')
ax[0].plot((18/20,16/20, 8/20), marker="v", color="green", label="高頻度")
ax[0].plot((14/20, 9/20, 3/20), marker="^", color="blue", label="低頻度")
ax[0].set_xlim(-0.5,2.5)
ax[0].set_ylim(0,1)
ax[0].set_xticks(ticks=range(3))
ax[0].set_xticklabels(labels=['一貫','非一貫','例外'])
ax[0].legend()
ax[0].set_title('単語')
ax[0].set_ylabel('正解率')

ax[1].plot((17/20,17/20,13/20), marker="v", color="green", label="高頻度")
ax[1].plot((15/20,13/20, 7/20), marker="^", color="blue", label="低頻度")
ax[1].set_xlim(-0.5,2.5)
ax[1].set_ylim(0,1)
ax[1].set_xticks(ticks=range(3))
ax[1].set_xticklabels(labels=['一貫','非一貫','例外'])
ax[1].set_title('非単語')
ax[1].legend()
fig.savefig('2023_0123LAM_o2p_hid256_fushimi1999.pdf')
plt.show()

# 1. 必要となるライブラリのインストール

注意: Colab 上で実行する場合，Google Drive への接続許可を求めるポップアップウィンドウが開くので，許可する必要があります。

In [ ]:
%config InlineBackend.figure_format = 'retina'

import torch
from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

if isColab:

    # termcolor を downgrade しないと colab ではテキストに色がつかない
    !pip install --upgrade termcolor==1.1
    import termcolor    

    # 結果を保存するために Google Drive をマウントする
    import google.colab
    google.colab.drive.mount('/content/drive/')
    
    # GPU 情報を表示
    !nvidia-smi -L

    #!pip install ipynbname --upgrade > /dev/null

if isColab:
    # colab 上で MeCab を動作させるために，C コンパイラを起動して，MeCab の構築を行う
    # そのため時間がかかる。
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    
    import MeCab
    mecab_wakati = MeCab.Tagger('-Owakati').parse
    mecab_yomi = MeCab.Tagger('-Oyomi').parse
    
else:
    from ccap.mecab_settings import yomi as mecab_yomi
    from ccap.mecab_settings import wakati as mecab_wakati


# ここから下は，コード実行に関するバージョン情報などの情報源の取得と表示
from termcolor import colored

import platform
HOSTNAME = platform.node().split('.')[0]

import os
HOME = os.environ['HOME']

try:
    import ipynbname
except ImportError:
    !pip install ipynbname
    import ipynbname
FILEPATH = str(ipynbname.path()).replace(HOME+'/','')

import pwd
USER=pwd.getpwuid(os.geteuid())[0]

from datetime import date
TODAY=date.today()

import torch
TORCH_VERSION = torch.__version__

color = 'green'
print('日付:',colored(f'{TODAY}', color=color, attrs=['bold']))
print('HOSTNAME:',colored(f'{HOSTNAME}', color=color, attrs=['bold']))
print('ユーザ名:',colored(f'{USER}', color=color, attrs=['bold']))
print('HOME:',colored(f'{HOME}', color=color,attrs=['bold']))
print('ファイル名:',colored(f'{FILEPATH}', color=color, attrs=['bold']))
print('torch.__version__:',colored(f'{TORCH_VERSION}', color=color, attrs=['bold']))

# 2. パラメータ設定

語彙数を 10K 語から 20K 語に倍増しているのは，Fushimi1999 の語彙リストの未知語が存在したためである。

In [4]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# シミュレーションに必要なパラメータの設定
params = {
    'traindata_size':  20000,   # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    'epochs': 30,               # 学習のためのエポック数
    
    'hidden_size': 256,          # 中間層のニューロン数
    #'hidden_size': 128,         # 中間層のニューロン数
    
    'random_seed': 42,          # 乱数の種。ダグラス・アダムス著「銀河ヒッチハイカーズガイド」

    # 以下 `source` と `target` を定義することで，別の課題を実行可能
    'source': 'orth',          # ['orth', 'phon']
    'target': 'phon',          # ['orth', 'phon']

    'pretrained': False,          # True であれば訓練済ファイルを読み込む
    #'isTrain'   : True,          # True であれば学習する
    
    # 学習済のモデルパラメータを保存するファイル名
    #'path_saved': '2022_0607lam_o2p_hid32_vocab10k.pt', 
    'path_saved': '2023_0201triangle2023_o2p_hid256.pt',
    #'path_saved': False,                      # 保存しない場合
    
    'lr': 1e-4,                     # 学習率
    'dropout_p': 0.0,                 # ドロップアウト率
    'teacher_forcing_ratio': 0.5,     # 教師強制を行う確率
    'optim_func': torch.optim.Adam,   # 最適化アルゴリズム ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.AdamW']
    'loss_func' :torch.nn.NLLLoss(),  # 負の対数尤度損失 ['torch.nn.NLLLoss()', or 'torch.nn.CrossEntropyLoss()']
}

color = 'green'
for k, v in params.items():
    print(f'{k}:{colored(v, color=color, attrs=["bold"])}')

traindata_size:20000
epochs:30
hidden_size:256
random_seed:42
source:orth
target:phon
pretrained:False
path_saved:2023_0201triangle2023_o2p_hid256.pt
lr:0.0001
dropout_p:0.0
teacher_forcing_ratio:0.5
optim_func:<class 'torch.optim.adam.Adam'>
loss_func:NLLLoss()


# 3. ライブラリの輸入

In [5]:
import math
import random
import numpy as np
import time
from tqdm import tqdm

try:
    import jaconv
except ImportError:
    !pip install jaconv
    import jaconv

# 4. データ作成

## 4.1. psylex71_dataset と vdrj_dataset の定義

* Psylex71 は「NTT 日本語の語彙特性」頻度表であり，著作権上の問題があるため配布不可
* vdrj は松下言語学習ラボ，[日本語を読むための語彙データベース（研究用）](http://www17408ui.sakura.ne.jp/tatsum/database.html#vdrj) を加工して作成したデータである

RAM ディレクトリ直下に，`psylex71_data.gz`, `vdrj_data.gz` がある。
これらは，`RAM/make_psylex71_dict.py`, `RAM/make_vdrj_dict.py` を実行して作成されたデータファイルである。
ここでは，これらのデータファイルが作成済と仮定している。


In [26]:
!rm -rf RAM

In [27]:
if isColab:
    !git clone https://github.com/ShinAsakawa/RAM.git 

from RAM import convert_ids2tensor    
from RAM import calc_accuracy    
from RAM import asMinutes
from RAM import timeSince
from RAM import check_vals_performance
from RAM import evaluate
#from RAM import draw_word_char_histgram
from RAM.utils import _train
from RAM.utils import _fit
# #from RAM import evaluate
#from RAM import Onechar_dataset
from RAM.dataset import OneChar_Dataset

Cloning into 'RAM'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 31 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), 14.31 MiB | 7.74 MiB/s, done.


## 4.2. Fushimi, Ijuin, and Patterson(1999) データの読み込み

- Fushimi1999 データは検証データとして用いるため，最初に読み込む

In [ ]:
from RAM.fushimi1999 import _fushimi1999_list
from RAM.fushimi1999 import fushimi1999_dict
fushimi1999_list = _fushimi1999_list()
for k, v in fushimi1999_dict.items():
    print(k,v)

In [29]:
%reload_ext autoreload
%autoreload 2

import gzip
import json
import sys
import re

from RAM.dataset import Psylex71_Dataset
from RAM.dataset import VDRJ_Dataset

psylex71_dataset = Psylex71_Dataset(source=params['source'], target=params['target'])
vdrj_dataset = VDRJ_Dataset(source=params['source'], target=params['target'])

ds = psylex71_dataset


## 4.3 定義したデータセット psylex71 と vdrj を宣言

### 4.3.1 確認作業

In [ ]:
wrd = '蕎麦'
N = 1999
for ds in [psylex71_dataset, vdrj_dataset]:
    #print(str(ds))
    #print(ds.data_dict[N])
    inp_ids, tgt_ids = ds.__getitem__(N)
    print(ds.source_ids2tkn(inp_ids), ds.target_ids2tkn(tgt_ids))
    #print(ds.orth2info_dict[wrd])

## 4.4 データセットの頻度情報の視覚化

In [ ]:
from RAM.utils import draw_word_char_histgram
ds = psylex71_dataset
draw_word_char_histgram(_dict=ds.data_dict, key='phon', title='psylex71 phon', figsize2=(8,3))
draw_word_char_histgram(_dict=ds.data_dict, key='orth', title='psylex71 orth', figsize2=(8,3))

ds = vdrj_dataset
draw_word_char_histgram(_dict=ds.data_dict, key='phon', title='vdrj phon', figsize2=(8,3))
draw_word_char_histgram(_dict=ds.data_dict, key='orth', title='vdrj orth', figsize2=(8,3))

ds = psylex71_dataset

# 5. 一文字データセットによる予備検証

## 5.1. 自作ライブラリ triangle2023 のインポート

## 5.2 一文字データセットの読み込み

In [ ]:
from RAM.dataset import OneChar_Dataset
onechar_dataset = OneChar_Dataset(
    source=params['source'], 
    target=params['target'])
#onechar_dataset.data_dict

In [ ]:
#params

In [ ]:
print(onechar_dataset.source_list)
print(onechar_dataset.target_list)
print(onechar_dataset.data_dict)

In [ ]:
draw_word_char_histgram(_dict=onechar_dataset.data_dict, key='orth', figsize2=(6,4))
draw_word_char_histgram(_dict=onechar_dataset.data_dict, key='phon', figsize2=(6,3))
#print(onechar_dataset.data_dict[0])

## 5.3. 符号化器-復号化器モデルの定義

In [ ]:
%reload_ext autoreload
%autoreload 2

from RAM.model import EncoderRNN
from RAM.model import AttnDecoderRNN
from RAM import train_one_seq2seq
from RAM import train_epochs

ds = psylex71_dataset

encoder = EncoderRNN(
    n_inp=len(onechar_dataset.source_list),                # 符号化器への入力データ次元数の特徴数 (語彙数): int
    n_hid=params['hidden_size']).to(device)   # 符号化器の中間層数，埋め込みベクトルとして復号化器へ渡される次元数: int
                                              # 復号化器の出力層素子数は，入力層と同一であるので指定しない

decoder = AttnDecoderRNN(
    n_hid=params['hidden_size'],               # 復号化器の中間層次元数: int
    n_out=len(onechar_dataset.target_list),                 # 復号化器の出力層次元数，入力層の次元と等しいので入力層次元を指定せず: int
    dropout_p=params['dropout_p'],
    max_length=ds.target_maxlen).to(device)

## 5.4. 訓練用最適化関数の定義

In [ ]:
try:
    econder_optimizer
except:
    encoder_optimizer = params['optim_func'](encoder.parameters(), lr=params['lr'])
    
try:
    decoder_optimizer
except:
    decoder_optimizer = params['optim_func'](decoder.parameters(), lr=params['lr'])

## 5.5. 一文字データセットを用いた訓練

In [ ]:
# 新たな訓練を実行する場合には，損失関数を消去する必要があります
#del losses
params

In [ ]:
try:
    losses
except:
    losses = []

losses += train_epochs( 
    #epochs=200,
    #epochs=params['epochs'], 
    epochs=30,
    
    #lr=params['lr'],
    lr=0.001,

    encoder=encoder, decoder=decoder,  
    encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
    source_vocab=ds.source_list, target_vocab=ds.target_list,
    source_ids=ds.source, target_ids=ds.target,
    params=params,
    
    device=device,
    max_length=ds.target_maxlen,
    n_sample=0,
    teacher_forcing_ratio=params['teacher_forcing_ratio'],
    
    #train_dataset=train_dataset,
    train_dataset=onechar_dataset,
    
    #val_dataset=X_vals,
    val_dataset=None,
    #val_dataset={'val':onechar_dataset},
)

try:
    plt
except:
    import matplotlib.pyplot as plt
plt.plot(losses)    

## 2.3 Fushimi1999 データによる検証

In [ ]:
def check_fushimi1999_list(encoder=encoder,
                           decoder=decoder,
                           is_print:bool=False,
                          )->str:

    ret_msg = ""
    counter = 1
    for key, val in fushimi1999_dict.items():
        key_old = ""
        if key != key_old:
            key_old = key
            ret_msg += f'{key}:'
            if is_print:
                print(colored(f'{key}:', 'green', attrs=['bold']), end=" ")
        
        n_ok, n_all = 0, 0
        msg = ""
        for wrd in val:
            _src_ids = psylex71_dataset.source_tkn2ids(wrd) + [psylex71_dataset.source_list.index('<EOW>')]
            ans=evaluate(encoder=encoder, decoder=decoder,
                         input_ids=_src_ids,
                         max_length= psylex71_dataset.target_maxlen,
                         source_vocab=psylex71_dataset.source_list, 
                         target_vocab=psylex71_dataset.target_list,
                         device=device)
            
            res = "".join(p for p in ans[0][:-1])  # モデルからの戻り値を再構成
            if res == wrd:
                n_ok += 1
                n_all += 1

            counter =  1 if (counter % 10) == 0 else (counter + 1)
            _end = "\n" if counter==1 else ", "
            msg += f"{wrd}->/{res}/{_end}"
        
        if n_all != 0:
            ret_msg += f'{n_ok/n_all * 100:5.2f}%\n{msg}'
        else:
            ret_msg += f'{0:5.2f}%\n{msg}'
        if is_print:
            print(f'{n_ok/n_all * 100:5.2f}%\n{msg}')
            
    return ret_msg
            
print(check_fushimi1999_list())

In [ ]:
%reload_ext autoreload
%autoreload 2

from triangle2023.model import EncoderRNN
from triangle2023.model import AttnDecoderRNN

encoder = EncoderRNN(
    n_inp=len(psylex71_dataset.source_list),         # 符号化器への入力データ次元数の特徴数 (語彙数): int
    n_hid=params['hidden_size']).to(device)   # 符号化器の中間層数，埋め込みベクトルとして復号化器へ渡される次元数: int
                                              # 復号化器の出力層素子数は，入力層と同一であるので指定しない

decoder = AttnDecoderRNN(
    n_hid=params['hidden_size'],               # 復号化器の中間層次元数: int
    n_out=len(psylex71_dataset.target_list),          # 復号化器の出力層次元数，入力層の次元と等しいので入力層次元を指定せず: int
    dropout_p=params['dropout_p'],
    max_length=psylex71_dataset.target_maxlen).to(device)

# 6. 全データを用いた訓練

## 6.1. 全データを訓練データと検証データに分割

In [ ]:
P  = int(psylex71_dataset.__len__() * 0.9)
_P = psylex71_dataset.__len__() - P
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset=psylex71_dataset,
    lengths=(P, _P),
    generator=torch.Generator().manual_seed(params['random_seed']))

## 6.2. 訓練の実施

In [ ]:
del losses

In [ ]:
try:
    losses
except:
    losses = []
losses += train_epochs(
    epochs=5,
    lr=params['lr'],
    #lr=0.001,
    
    encoder=encoder, decoder=decoder, 
    encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
    source_vocab=psylex71_dataset.source_list, target_vocab=psylex71_dataset.target_list,
    source_ids=psylex71_dataset.source, target_ids=psylex71_dataset.target,
    params=params,
    device=device,
    max_length=psylex71_dataset.target_maxlen,
    n_sample=0,
    teacher_forcing_ratio=params['teacher_forcing_ratio'],
    train_dataset=train_dataset,
    val_dataset={'val': val_dataset},
)

try:
    plt
except:
    import matplotlib.pyplot as plt
plt.plot(losses)    

In [ ]:
pt_fname = '2023_0210fname.pt'
torch.save({'encoder':encoder.state_dict(), 'decoder':decoder.state_dict(),
            'encoder_optimizer': encoder_optimizer.state_dict(), 'decoder_optimizer': decoder_optimizer.state_dict(),
            'params': params, 'data': psylex71_dataset.data_dict}, 
           pt_fname)